---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [3]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [4]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [10]:
def graph_identification():
    result = []
    for G in P1_Graphs:
        degrees = G.degree()
        degree_values = sorted(set(degrees.values()))
        degree_hist = [list(degrees.values()).count(i) / float(nx.number_of_nodes(G)) for i in degree_values]
        clustering = nx.average_clustering(G)
        shortest_path = nx.average_shortest_path_length(G)
        
        if len(degree_hist)>10:# if power law, then there will be some weight at high values of the degree
            result.append('PA')
        elif clustering < 0.1:
            result.append('SW_H')
        else:
            result.append('SW_L')
    return result
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [71]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [76]:
def salary_predictions():
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score

    df = pd.DataFrame(index=G.nodes())
    df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
    df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
    df['Degree'] = pd.Series(G.degree())
    df['Clustering'] = pd.Series(nx.clustering(G))
    df['DegreeCentrality'] = pd.Series(nx.degree_centrality(G))
    df['BetweenessCentrality'] = pd.Series(nx.betweenness_centrality(G))
    df['ClosenessCentrality'] = pd.Series(nx.closeness_centrality(G))
    df['PageRank'] = pd.Series(nx.pagerank(G))
    X=df.dropna().drop(labels=['ManagementSalary'], axis=1)
    y=df.dropna().ManagementSalary
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    X_val=df[pd.isnull(df['ManagementSalary'])].drop(labels=['ManagementSalary'], axis=1)

    clf = RandomForestClassifier(n_estimators=100, max_depth=10).fit(X_train, y_train)
    pred = clf.predict(X_test) 
    pred_proba = clf.predict_proba(X_test)[:, 1]
    #print(roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]))

    pred = clf.predict(X_val) 
    pred_proba = clf.predict_proba(X_val)[:, 1]
    
    return pd.Series(pred_proba, index=X_val.index)# Your Answer Here

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [51]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [67]:
def new_connections_predictions():
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import roc_auc_score
    
    df=future_connections
    df['Origin'] = [orig for (orig, dest) in df.index]
    df['Destiny'] = [dest for (orig, dest) in df.index]
    df['Origin degree'] = [G.degree(node) for node in df['Origin']]
    df['Destiny degree'] = [G.degree(node) for node in df['Destiny']]
    df['Community'] = [G.nodes(data=True)[node][1]['Department'] for node in df['Origin']]
    df['Edge'] = df.index
    df['Preferential Attachment'] = [list(nx.preferential_attachment(G, [edge]))[0][2] for edge in df['Edge']]
    df['Common Neighbors'] = [list(nx.common_neighbors(G, edge[0], edge[1])) for edge in df['Edge']]
    df['Num of Common Neighbors'] = [len(neighbors) for neighbors in df['Common Neighbors']]
    df['Jaccard Coefficient'] = [list(nx.jaccard_coefficient(G, [edge]))[0][2] for edge in df['Edge']]
    df['Resource Allocation Index'] = [list(nx.resource_allocation_index(G, [edge]))[0][2] for edge in df['Edge']]
    df['Adamic Adar Index'] = [list(nx.adamic_adar_index(G, [edge]))[0][2] for edge in df['Edge']]
    df['CN Soundarajan Hopcroft'] = [list(nx.cn_soundarajan_hopcroft(G, ebunch=[edge], community='Department'))[0][2] for edge in df['Edge']]
    df['RA Soundarajan Hopcroft'] =  [list(nx.ra_index_soundarajan_hopcroft(G, ebunch=[edge], community='Department'))[0][2] for edge in df['Edge']]
    
    features = ['Origin degree', 'Destiny degree', 'Community', 'Preferential Attachment',
                'Num of Common Neighbors', 'Jaccard Coefficient', 'Resource Allocation Index', 'Adamic Adar Index',
                'CN Soundarajan Hopcroft', 'RA Soundarajan Hopcroft']
    df1=df.dropna()
    X=df1[features]
    y=df1['Future Connection']
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    X_val=df[pd.isnull(df['Future Connection'])].drop(labels=['Future Connection'], axis=1)

    clf = RandomForestClassifier(n_estimators=100, max_depth=10).fit(X_train, y_train)
    pred = clf.predict(X_test) 
    pred_proba = clf.predict_proba(X_test)[:, 1]
    #print(roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]))

    pred = clf.predict(X_val[features]) 
    pred_proba = clf.predict_proba(X_val[features])[:, 1]
    
    return pd.Series(pred_proba, index=X_val.index)# Your Answer Here

new_connections_predictions()

(107, 348)    0.019418
(542, 751)    0.041073
(20, 426)     0.605449
(50, 989)     0.007528
(942, 986)    0.007764
(324, 857)    0.009448
(13, 710)     0.101580
(19, 271)     0.157081
(319, 878)    0.008443
(659, 707)    0.007676
(49, 843)     0.007652
(208, 893)    0.007531
(377, 469)    0.009407
(405, 999)    0.010592
(129, 740)    0.038335
(292, 618)    0.057203
(239, 689)    0.008443
(359, 373)    0.009961
(53, 523)     0.419466
(276, 984)    0.041737
(202, 997)    0.041704
(604, 619)    0.034577
(270, 911)    0.041817
(261, 481)    0.055948
(200, 450)    0.984049
(213, 634)    0.009630
(644, 735)    0.026773
(346, 553)    0.012133
(521, 738)    0.012417
(422, 953)    0.010665
                ...   
(672, 848)    0.009394
(28, 127)     0.989968
(202, 661)    0.045041
(54, 195)     0.999989
(295, 864)    0.008586
(814, 936)    0.013728
(839, 874)    0.012877
(139, 843)    0.009491
(461, 544)    0.009257
(68, 487)     0.009387
(622, 932)    0.007554
(504, 936)    0.012487
(479, 528) 